In [1]:
#Necessary imports
import pandas as pd
import numpy as np
from tqdm import tqdm
import tweepy
from datetime import date
import pickle 
import time
import matplotlib.pyplot as plt

## Subset data

In [284]:
tweets = pd.read_csv("mp_tweets", index_col=0, low_memory=False)

In [285]:
tweets = tweets.rename(columns={"user_id":"m.uid"})

In [286]:
remove = [175522, 190414, 211953, 212012, 212013, 212298 ]

In [287]:
tweets = tweets.drop(tweets.index[remove])

In [288]:
tweets = tweets.dropna(subset=["screen_name"])

In [289]:
tweets["created_at"] = pd.to_datetime(tweets["created_at"])

### MP info

In [290]:
#Filter by Australian Parlamentarians
#Download the file full_member_info at http://twitterpoliticians.org/download
mp_df = pd.read_csv("data/full_member_info.csv")
mp_df = mp_df[['p.country', 'm.name', 'p.party', 'm.uid', 'lp.official_legislative_period']].copy()
mp_df = mp_df.loc[mp_df["p.country"]=="Australia"]
mp_df["m.name"] = mp_df["m.name"].str.replace(r"^Hon\s{1}|^Mr\s{1}|\s{1}MP", "")
mp_info =  mp_df.loc[mp_df["m.uid"] != "\\N"]
mp_info["m.uid"] = mp_info["m.uid"].astype(int)

<ipython-input-290-203387997bde>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  mp_df["m.name"] = mp_df["m.name"].str.replace(r"^Hon\s{1}|^Mr\s{1}|\s{1}MP", "")
<ipython-input-290-203387997bde>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mp_info["m.uid"] = mp_info["m.uid"].astype(int)


In [291]:
tweets["m.uid"] = tweets["m.uid"].astype(int)

In [292]:
tweets = tweets.merge(mp_info, on = "m.uid", how = "left")

In [293]:
tweets

,screen_name,m.uid,tweet_id,created_at,full_text,favorite_count,retweet_count,in_reply_to_screen_name,hashtags,user_mentions,url,image_url,p.country,m.name,p.party,lp.official_legislative_period
0,AlanTudgeMP,185932331,1.389486e+18,2021-05-04 07:42:40,Research commercialisation in action. @QUT wor...,25,5.0,NaN,[],"['QUT', 'TrevorEvansBne']",NaN,NaN,Australia,Alan Tudge,Liberal Party of Australia,45
1,AlanTudgeMP,185932331,1.389486e+18,2021-05-04 07:42:40,Research commercialisation in action. @QUT wor...,25,5.0,NaN,[],"['QUT', 'TrevorEvansBne']",NaN,NaN,Australia,Alan Tudge,Liberal Party of Australia,46
2,AlanTudgeMP,185932331,1.389434e+18,2021-05-04 04:18:43,Hearing first hand how our extra $1.7b boost t...,7,0.0,NaN,[],"['GoodStartel', 'BertVanManen']",NaN,NaN,Australia,Alan Tudge,Liberal Party of Australia,45
3,AlanTudgeMP,185932331,1.389434e+18,2021-05-04 04:18:43,Hearing first hand how our extra $1.7b boost t...,7,0.0,NaN,[],"['GoodStartel', 'BertVanManen']",NaN,NaN,Australia,Alan Tudge,Liberal Party of Australia,46
4,AlanTudgeMP,185932331,1.389430e+18,2021-05-04 04:01:51,RT @CISOZ: [NEW EDUCATION EVENT]\nTuesday 18 M...,0,5.0,NaN,[],"['CISOZ', 'AlanTudgeMP']",NaN,NaN,Australia,Alan Tudge,Liberal Party of Australia,45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
575232,P_Thompson88,929896339307053056,9.302562e+17,2017-11-14 02:08:55,A #LivedExperiencedVeteran voice at the macro ...,0,0.0,NaN,"['LivedExperiencedVeteran', 'SuicidePrevention...",[],NaN,NaN,Australia,Phillip Thompson,Liberal National Party of Queensland,46
575233,P_Thompson88,929896339307053056,9.302450e+17,2017-11-14 01:24:36,#NewProfilePic https://t.co/jVWltJHHRD,1,0.0,NaN,['NewProfilePic'],[],NaN,NaN,Australia,Phillip Thompson,Liberal National Party of Queensland,46
575234,P_Thompson88,929896339307053056,9.298999e+17,2017-11-13 02:33:07,Very honoured to be named the 2018 QLD Young A...,13,6.0,NaN,"['SuicidePrevention', 'veteransuicidepreventio...",[],NaN,NaN,Australia,Phillip Thompson,Liberal National Party of Queensland,46
575235,P_Thompson88,929896339307053056,9.298993e+17,2017-11-13 02:30:38,RT @ausoftheyear: A young veteran’s champion d...,0,5.0,NaN,['QLD'],['ausoftheyear'],NaN,NaN,Australia,Phillip Thompson,Liberal National Party of Queensland,46


In [131]:
#tweets["created_at"] = pd.to_datetime(tweets["created_at"], format="%Y-%m-%d")

In [296]:
subset = tweets.loc[((tweets["lp.official_legislative_period"] == "45") & (tweets["created_at"] < "2019-07-01"))|
                    ((tweets["lp.official_legislative_period"] == "46") & (tweets["created_at"] > "2019-07-01")) ]

In [299]:
len(subset["tweet_id"].unique())

299139

In [302]:
subset.to_csv("tweets_subset")